# Projeto - Crédito para Financiamento de Imóveis

## Orientações gerais

O projeto de encerramento de curso será dividido em três partes e utilizado como avaliação dos módulos de Data Engineering, Data Science e AWS.

## Contextualização

A PyCoders Ltda., cada vez mais especializada no mundo da Inteligência Artificial e Ciência de Dados, foi procurada por uma fintech para desenvolver um projeto de concessão de crédito para imóveis. Nesse projeto, espera-se a criação de score que discrimine ao máximo os bons pagadores dos maus pagadores. Para isso, foi disponibilizada uma base de dados com milhares de casos de empréstimos do passado, com diversas características dos clientes. Deve ser entregue um modelo para realizar essa classificação. Por questões contratuais, o pagamento será realizado baseado no desempenho (ROC AUC).


## Base de Dados

Serão utilizadas bases de dados com informações cadastrais, histórico de crédito e balanços financeiros de diversos clientes. O conjunto de dados está dividido em treino e teste, todos no formato csv. Toda a modelagem, validação e avaliação deve ser feita em cima do conjunto de treino, subdividindo tal base como achar melhor. Existe também a base das variáveis explicativas, para ajudar no desenvolvimento do projeto.

[Baixar aqui](https://s3-sa-east-1.amazonaws.com/lcpi/0694c90a-7782-47f7-8bbc-e611d31f9f21.zip)



# Parte 1: Data Engineering

**Preparação:** Salve os arquivos `.csv` em `/FileStore/tables/projeto_credito/`, sem alterar seus nomes.

## Antes de modelar

1. Crie um `fluxo de dados` no Databricks para as bases que serão utiizadas.
    1. Insira os dados brutos na primeira camada.
    1. Salve as transformações / limpezas na segunda camada.
   
    1. Crie uma pipeline para o processo.
        1. Pipeline, nesse caso, será um único script que gere todas as tabelas acima.

> ### Resolução
> 
> Upload das tabelas no `/FileStore/tables/projeto_credito/` do Databricks:
> 
> ![Upload das tabelas](./imgs/01.jpg)
> 
> Link para `notebook` do Databricks com a pipeline dessa etapa: [aqui](https://databricks-prod-cloudfront.cloud.databricks.com/public/4027ec902e239c93eaaa8714f173bcfc/3711839099470423/2262021360226139/5022278466657041/latest.html).

## Durante a modelagem

1. Selecione e salve as colunas relevantes e features criadas na terceira camada.

1. Salve as versões do modelo no formato `pickle` no DBFS.

1. Mantenha o controle das versões criando uma tabela no formato
**ATENÇÂO:** Como estamos na versão community, lembre de exportar a tabela abaixo para o DBFS antes da sessão encerrar.

|id_modelo|nome_modelo|data_treino|método|roc_auc|tempo_de_treino(s)|hyperparametros|path_to_pickle
|---|---|---|---|---|---|---|---|
|1|RandomForestRapida|2022-02-22|Random Forest Simples|0.76|124|\[max_depth=4, ...\]|/FileStore/tables/modelos/...|

## Após modelar

1. Salve os dados de treino e validação em uma tabela (para que seja possível reproduzir resultados no futuro).
    1. Crie uma coluna com o id do modelo escolhido para refêrencia. 

## Regras de Entrega

1. Um notebook (databricks) com a pipeline que gere as tabelas da fase *Antes de modelar*. O notebook deve rodar de uma vez só!

1. Um arquivo `.csv` com as informações da tabela gerada na fase *Durante a modelagem*.

1. Um notebook que gere as tabelas de treino e validação do passo *Após modelar*. O notebook deve rodar de uma vez só!

> **IMPORTANTE:** Tendo em vista que não teremos apresentação do projeto (e não queromos pedir que vocês gravem um vídeo explicando o notebook, haha), é indispensável que ele esteja organizado e comentado.

In [1]:
import pandas as pd

In [2]:
df_train = pd.read_csv('./data/application_train.csv')
df_train.head()


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,456162,0,Cash loans,F,N,N,0,112500.0,700830.0,22738.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,134978,0,Cash loans,F,N,N,0,90000.0,375322.5,14422.5,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,3.0
2,318952,0,Cash loans,M,Y,N,0,180000.0,544491.0,16047.0,...,0,0,0,0,0.0,0.0,0.0,1.0,1.0,3.0
3,361264,0,Cash loans,F,N,Y,0,270000.0,814041.0,28971.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
4,260639,0,Cash loans,F,N,Y,0,144000.0,675000.0,21906.0,...,0,0,0,0,0.0,0.0,0.0,10.0,0.0,0.0


In [3]:
df_train['NAME_CONTRACT_TYPE'].value_counts()

Cash loans         222622
Revolving loans     23386
Name: NAME_CONTRACT_TYPE, dtype: int64

In [4]:
df_train.columns

Index(['SK_ID_CURR', 'TARGET', 'NAME_CONTRACT_TYPE', 'CODE_GENDER',
       'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL',
       'AMT_CREDIT', 'AMT_ANNUITY',
       ...
       'FLAG_DOCUMENT_18', 'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20',
       'FLAG_DOCUMENT_21', 'AMT_REQ_CREDIT_BUREAU_HOUR',
       'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK',
       'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT',
       'AMT_REQ_CREDIT_BUREAU_YEAR'],
      dtype='object', length=122)

In [6]:
df_test = pd.read_csv('./data/application_test_student.csv')
df_test.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,149741,Cash loans,F,N,N,0,117000.0,417024.0,20191.5,360000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
1,363290,Cash loans,M,N,Y,0,450000.0,640080.0,31261.5,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
2,436006,Revolving loans,M,Y,Y,0,450000.0,900000.0,45000.0,900000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,2.0,0.0
3,377703,Cash loans,M,Y,N,1,360000.0,1125000.0,33025.5,1125000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
4,188624,Cash loans,M,Y,Y,0,675000.0,835380.0,42840.0,675000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0


In [7]:
df_train_cols = df_train.columns
df_test_cols = df_test.columns
df_train_cols.shape

(122,)

In [8]:
df_cols = pd.read_csv('./data/HomeCredit_columns_description.csv', encoding= 'unicode_escape', index_col=0)
df_cols = df_cols[df_cols['Table'] == 'application_{train|test}.csv']
df_cols = df_cols[['Row', 'Description']]
df_cols.head()

,Row,Description
1,SK_ID_CURR,ID of loan in our sample
2,TARGET,Target variable (1 - client with payment diffi...
5,NAME_CONTRACT_TYPE,Identification if loan is cash or revolving
6,CODE_GENDER,Gender of the client
7,FLAG_OWN_CAR,Flag if the client owns a car


In [9]:
# Função auxiliar para auxiliar na análise das colunas
def search_definition(dataframe):
    column_name = input('Digite o nome da coluna (0 para sair): ')
    definition = dataframe[dataframe['Row'] == column_name]['Description']
    print(definition.values[0])

In [10]:
search_definition(df_cols)

KeyboardInterrupt: Interrupted by user

In [ ]:
df_cleaned = df_train.copy()

In [49]:
colunas_inutilizadas = [
    'SK_ID_CURR',
    'NAME_TYPE_SUITE',
    'OWN_CAR_AGE',
    'FLAG_EMP_PHONE',
    'FLAG_WORK_PHONE',
    'FLAG_CONT_MOBILE',
    'FLAG_PHONE',
    'NAME_TYPE_SUITE',
    'COMMONAREA_AVG',
    'ELEVATORS_AVG',
    'ENTRANCES_AVG',
    'FLOORSMAX_AVG',
    'FLOORSMIN_AVG',
    'LANDAREA_AVG',
    'LIVINGAPARTMENTS_AVG',
    'LIVINGAREA_AVG',
    'NONLIVINGAPARTMENTS_AVG',
    'NONLIVINGAREA_AVG',
    'LANDAREA_MEDI',
    'LIVINGAPARTMENTS_MEDI',
    'LIVINGAREA_MEDI',
    'NONLIVINGAPARTMENTS_MEDI',
    'NONLIVINGAREA_MEDI',
    'NONLIVINGAPARTMENTS_MODE',
    'NONLIVINGAREA_MODE',
    'FONDKAPREMONT_MODE',
    'APARTMENTS_MODE',
    'BASEMENTAREA_MODE',
    'YEARS_BEGINEXPLUATATION_MODE',
    'YEARS_BUILD_MODE',
    'COMMONAREA_MODE',
    'ELEVATORS_MODE',
    'ENTRANCES_MODE',
    'FLOORSMAX_MODE',
    'FLOORSMIN_MODE',
    'LANDAREA_MODE',
    'LIVINGAPARTMENTS_MODE',
    'LIVINGAREA_MODE',
    'NONLIVINGAPARTMENTS_MODE',
    'NONLIVINGAREA_MODE',
    'FONDKAPREMONT_MODE',
    'WALLSMATERIAL_MODE',
    'EMERGENCYSTATE_MODE',
    'FLAG_DOCUMENT_2',
    'FLAG_DOCUMENT_3',
    'FLAG_DOCUMENT_4',
    'FLAG_DOCUMENT_5',
    'FLAG_DOCUMENT_6',
    'FLAG_DOCUMENT_7',
    'FLAG_DOCUMENT_8',
    'FLAG_DOCUMENT_9',
    'FLAG_DOCUMENT_10',
    'FLAG_DOCUMENT_11',
    'FLAG_DOCUMENT_12',
    'FLAG_DOCUMENT_13',
    'FLAG_DOCUMENT_14',
    'FLAG_DOCUMENT_15',
    'FLAG_DOCUMENT_16',
    'FLAG_DOCUMENT_17',
    'FLAG_DOCUMENT_18',
    'FLAG_DOCUMENT_19',
    'FLAG_DOCUMENT_20',
    'FLAG_DOCUMENT_21'
]

In [14]:
for column in df_train.columns:
    print("COLUMN:", column)
    print(df_cols[df_cols['Row'] == column]['Description'].values[0])
    print("\n")

ID of loan in our sample


Target variable (1 - client with payment difficulties: he/she had late payment more than X days on at least one of the first Y installments of the loan in our sample, 0 - all other cases)


Identification if loan is cash or revolving


Gender of the client


Flag if the client owns a car


Flag if client owns a house or flat


Number of children the client has


Income of the client


Credit amount of the loan


Loan annuity


For consumer loans it is the price of the goods for which the loan is given


Who was accompanying client when he was applying for the loan


Clients income type (businessman, working, maternity leave,)


Level of highest education the client achieved


Family status of the client


What is the housing situation of the client (renting, living with parents, ...)


Normalized population of region where client lives (higher number means the client lives in more populated region)


Client's age in days at the time of application


How many

# Parte 2: Data Science

## Requisitos Obrigatórios do Projeto

1. **Análise Exploratória dos Dados:** análise descritiva dos dados numéricos e categóricos, bem como gráficos (de sua preferência).
2. **Data Cleaning:** a base de dados apresenta dados ausentes. Sendo assim, você deverá realizar uma limpeza dos dados, removendo-os ou preenchendo com valores coerentes.
3. **Conversão de variáveis categóricas**
4. **Balanceamento de amostras:** nesse caso, como o dataset possui muitas amostras, você pode utilizar o NearMiss para realizar um *under sampling*.
5. **Machine Learning:** aplique algum algoritmo de ML, de sua preferência, dividindo o seu conjunto de dados em treino e teste, para obter o `roc_auc_score` de ambos os cenários (treino e teste).

## Regras de Entrega

1. Deve ser entregue uma base com as predições para a base de teste.
    - Essa base deverá ser um Data Frame com duas colunas: a primeira sendo o SK_ID_CURR e a segunda a probabilidade de inadimplência.
    - ⚠️ Entregar as predições com a probabilidade da inadimplência ocorrer.
2. Deve ser entregue o notebook com as etapas que foram aplicadas na criação do modelo (especificadas na subseção anterior).

> **IMPORTANTE:** Tendo em vista que não teremos apresentação do projeto (e não queromos pedir que vocês gravem um vídeo explicando o notebook, haha), é indispensável que ele esteja organizado e comentado.

<a href="https://s3-sa-east-1.amazonaws.com/lcpi/94acac51-8ce4-465b-a06d-a1cf19ec5d93.ipynb" style="display: block; background-color: #222; padding: 20px; text-align: center; font-weight: 600;">
Clique aqui para fazer o download do notebook com as instruções.
</a>

# Parte 3: AWS

**TBD**